In [1]:
import pandas as pd
import numpy as np

PART A

In [3]:
cols = ["Sex","Length","Diameter","Height","Whole_weight",
        "Shucked_weight","Viscera_weight","Shell_weight","Rings"]

df = pd.read_csv("abalone.data", names=cols)

In [4]:
print("Rows:", len(df))

Rows: 4177


In [5]:
print("Columns:", df.columns.tolist())

Columns: ['Sex', 'Length', 'Diameter', 'Height', 'Whole_weight', 'Shucked_weight', 'Viscera_weight', 'Shell_weight', 'Rings']


In [6]:
print(df.head())

  Sex  Length  Diameter  Height  Whole_weight  Shucked_weight  Viscera_weight  \
0   M   0.455     0.365   0.095        0.5140          0.2245          0.1010   
1   M   0.350     0.265   0.090        0.2255          0.0995          0.0485   
2   F   0.530     0.420   0.135        0.6770          0.2565          0.1415   
3   M   0.440     0.365   0.125        0.5160          0.2155          0.1140   
4   I   0.330     0.255   0.080        0.2050          0.0895          0.0395   

   Shell_weight  Rings  
0         0.150     15  
1         0.070      7  
2         0.210      9  
3         0.155     10  
4         0.055      7  


In [7]:
# Input: numeric body measurements
# Output: Rings+1.5 (age approx)
# Output numeric because regression problem

In [8]:
# Target
df["y"] = df["Rings"] + 1.5

In [9]:
features = ["Length", "Diameter", "Whole_weight"]

In [10]:
#Length:
#Represents overall shell size, which usually increases with age.
#Larger length generally indicates an older abalone.

#Diameter:
#Shows shell width and growth pattern along with length.
#Helps capture body proportion related to age.

#Whole_weight:
#Reflects total body mass including shell and meat.
#Weight typically increases as abalone grows older.

In [11]:
X = df[features].values
y = df["y"].values.reshape(-1,1)

In [12]:
n = len(X)
split = int(0.8*n)

X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Train shape: (3341, 3)
Test shape: (836, 3)


In [13]:
#Normalising
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train = (X_train-mean)/std
X_test = (X_test-mean)/std

In [14]:
# Normalisation is needed because features have different scales;
# normalization helps faster stable learning.

PART B

In [15]:
def forward(X, w, b):
    y_hat = X @ w + b
    print("Shapes -> X:",X.shape,"w:",w.shape,"b:",np.shape(b),"y_hat:",y_hat.shape)
    return y_hat

PART C

In [16]:
def mse(y, y_hat):
    return np.mean((y-y_hat)**2)


In [17]:
# Squaring penalizes large errors strongly.
# Big mistakes become expensive.

PART D

In [18]:
def grad_w(X, y, y_hat):
    N = len(y)
    return (2/N) * X.T @ (y_hat-y)

In [19]:
def grad_b(y, y_hat):
    N = len(y)
    return (2/N) * np.sum(y_hat-y)

In [20]:
# Gradient = slope of loss wrt parameters.
# Subtracting gradient reduces loss.
# Large gradient -> steep slope.
# Too big LR -> unstable training.

PART E

In [21]:
np.random.seed(0)
d = X_train.shape[1]

w = np.random.randn(d,1)*0.01
b = 0.0

lr = 0.01
epochs = 200

In [22]:
for epoch in range(epochs):
    y_hat = X_train @ w + b
    loss = mse(y_train, y_hat)

    dw = grad_w(X_train, y_train, y_hat)
    db = grad_b(y_train, y_hat)

    w -= lr*dw
    b -= lr*db

    if epoch%20==0:
        print("Epoch",epoch,"Loss",loss)


Epoch 0 Loss 144.18425894307313
Epoch 20 Loss 67.16262706192053
Epoch 40 Loss 33.96400741485028
Epoch 60 Loss 19.270563190355205
Epoch 80 Loss 12.730140339438332
Epoch 100 Loss 9.81480397744807
Epoch 120 Loss 8.514390424428589
Epoch 140 Loss 7.9337003717691
Epoch 160 Loss 7.673809434028033
Epoch 180 Loss 7.556921848903704


In [23]:
# Initial expectation: slow decrease
# After training: loss decreases steadily.


PART F

In [24]:
y_pred = X_test @ w + b

test_mse = mse(y_test, y_pred)
test_mae = np.mean(np.abs(y_test-y_pred))

print("Test MSE:", test_mse)
print("Test MAE:", test_mae)

print("\nSample predictions:")
for i in range(5):
    print("True:", y_test[i,0],
          "Pred:", y_pred[i,0],
          "Error:", abs(y_test[i,0]-y_pred[i,0]))

Test MSE: 5.162095323361795
Test MAE: 1.7153683752226243

Sample predictions:
True: 13.5 Pred: 10.781226334330654 Error: 2.718773665669346
True: 15.5 Pred: 9.724156106617547 Error: 5.7758438933824525
True: 14.5 Pred: 10.179004529174053 Error: 4.320995470825947
True: 14.5 Pred: 10.638206159657457 Error: 3.8617938403425427
True: 13.5 Pred: 10.652340427785969 Error: 2.847659572214031


In [25]:
# Systematic error often for very young/old abalones.
# Slight regression bias toward mean age.